In [1]:
import glob
from itertools import product

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sourmash import signature as sig
from tqdm import tqdm

%matplotlib inline

## Info about the kmer hashes loaded

In [2]:
ksizes = 21, 27, 33, 51
ksize = ','.join(map(str, ksizes))
num_hashes = int(1e6)
dna = True
protein = True


In [3]:
output_prefix = 's3://olgabot-maca/facs/sourmash_compute_tissue_subset/'


In [4]:
! aws s3 ls $output_prefix | wc -l

1000


### Copy the files to ndnd

In [6]:
! aws s3 sync $output_prefix /home/olga/scratch/maca/facs/sourmash_compute_tissue_subset/

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A14-B002775-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A14-B002775-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A14-MAA000801-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A14-MAA000801-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A14-MAA000487-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A14-MAA000487-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A14-MAA001857-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A14-MAA001857-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A14-MAA001862-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A14-MAA001862-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A15-MAA000776-3_10_M-1-1.

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A6-D042253-3_9_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A6-D042253-3_9_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A6-MAA100042-3_9_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A6-MAA100042-3_9_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A7-D042253-3_9_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A7-D042253-3_9_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A7-MAA000508-3_9_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A7-MAA000508-3_9_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A7-MAA000577-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A7-MAA000577-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A7-MAA000801-3_11_M-1-1.sig to ../../../scratch/m

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C13-MAA000801-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C13-MAA000801-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C14-D041914-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C14-D041914-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C14-MAA000487-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C14-MAA000487-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C15-MAA000574-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C15-MAA000574-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C14-MAA000776-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C14-MAA000776-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C15-MAA000577-3_8_M-1-1.sig t

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C9-MAA000779-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C9-MAA000779-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C9-MAA000907-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C9-MAA000907-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C9-MAA100042-3_9_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C9-MAA100042-3_9_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/D1-B000610-3_56_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/D1-B000610-3_56_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/D1-B000971-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/D1-B000971-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/D1-MAA000545-3_8_M-1-1.sig to ../../../sc

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/E17-MAA001862-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/E17-MAA001862-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/E18-B002764-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/E18-B002764-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/E18-MAA000487-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/E18-MAA000487-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/E19-MAA000577-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/E19-MAA000577-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/E18-MAA000776-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/E18-MAA000776-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/E19-MAA000779-3_11_M-1-1.si

...

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/L21-MAA001862-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/L21-MAA001862-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/L3-B001717-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/L3-B001717-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/L22-MAA000487-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/L22-MAA000487-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/L3-B002775-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/L3-B002775-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/L3-MAA000559-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/L3-MAA000559-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/L4-MAA000487-3_10_M-1-1.sig to ../../

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/M18-B000971-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/M18-B000971-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/M18-B001750-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/M18-B001750-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/M18-MAA000907-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/M18-MAA000907-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/M18-MAA000779-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/M18-MAA000779-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/M19-B000971-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/M19-B000971-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/M19-D042253-3_9_M-1-1.sig to ../.

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/N12-MAA000779-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/N12-MAA000779-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/N12-MAA000801-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/N12-MAA000801-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/N13-B001717-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/N13-B001717-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/N13-MAA000545-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/N13-MAA000545-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/N13-MAA000884-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/N13-MAA000884-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/N14-B002771-3_39_F-1-1.sig 

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/O21-MAA000559-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/O21-MAA000559-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/O22-B000971-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/O22-B000971-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/O22-MAA001857-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/O22-MAA001857-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/O3-MAA000907-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/O3-MAA000907-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/O4-B002771-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/O4-B002771-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/O4-MAA000487-3_10_M-1-1.sig to ../.

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/P18-MAA000776-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/P18-MAA000776-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/P18-MAA000907-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/P18-MAA000907-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/P19-B002775-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/P19-B002775-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/P19-MAA001857-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/P19-MAA001857-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/P2-B000610-3_56_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/P2-B000610-3_56_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/P20-D041914-3_8_M-1-1.sig to ..

## Read signatures

In [7]:
range(3)

range(0, 3)

In [ ]:

folder = '/home/olga/scratch/maca/facs/sourmash_compute_tissue_subset/'

filenames = glob.glob(f'{folder}/*.sig')


siglist = []

moltype = 'DNA'
ksize = 21


for filename in tqdm(filenames):
    loaded = sig.load_signatures(filename)
    siglist.extend(loaded)

 26%|██▌       | 260/1000 [17:50:39<55:52:52, 271.86s/it]

In [ ]:
%%time

import pickle

pickle_filename = f'{folder}/siglist.pickle'

with open(pickle_filename, 'wb') as f:
    pickle.dump(siglist, f)

In [ ]:
range(3)

In [ ]:
ls -lha $pickle_filename